<a href="https://colab.research.google.com/github/NicoMedinaa/LaboratorioVentas/blob/main/Laboratorio_de_An%C3%A1lisis_de_ventas_mensuales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importo Librerias


In [ ]:
import pandas as pd
import os
import plotly.graph_objects as go

# Cargo el dataframe y lo limpio

In [ ]:
rutaDeAcceso = "/content/drive/MyDrive/Dataset de ventas2"

archivos = os.listdir(rutaDeAcceso) #Creo un dataframe vacio para poder unir todos los dataframes

dataframes = {}
for archivo in archivos: #Recorro todos los archivos que se encuentran en la carpeta
    if archivo.endswith('.csv'):  # Asegurarse de que solo se consideren archivos CSV
        ruta_completa = os.path.join(rutaDeAcceso, archivo)  # Ruta completa de cada archivo individual
        mes = archivo.split("_")[3].split(".")[0]
        df = pd.read_csv(ruta_completa) #Guarda el dataframe del mes

        df['Cantidad Pedida'] = pd.to_numeric(df['Cantidad Pedida'], errors='coerce')  #El errors= 'coerce' transforma los valores no numericos a NaN'
        df['Precio Unitario'] = pd.to_numeric(df['Precio Unitario'], errors='coerce')

        df = df.loc[~df['Cantidad Pedida'].isna()]
        df = df.loc[~df['Precio Unitario'].isna()]

        # Elimino todos los Nan y me quedo solo con los valores numericos
        df = df.dropna()

        # Transformo todos los valores numericos al tipo de dato int
        df['Cantidad Pedida'] = df['Cantidad Pedida'].astype(int)
        df['Precio Unitario'] = df['Precio Unitario'].astype(int)

        dataframes[mes] = df #Y loego lo concateno con el resto de los dataframes
dataframes #Como resultado tengo un diccionario con cada mes como key y el value es el dataframe de ventas de ese mes

# Creo un dataframe general

In [ ]:
meses = ["Enero", "Febrero", "Marzo", "Abril", "Mayo", "Junio", "Julio", "Agosto", "Septiembre", "Octubre", "Noviembre", "Diciembre"]
#Creo una lista de messes
dfOrdenado = pd.DataFrame() #Creo un dataframe vacio

for mes in meses: #Recorro por cada mes
  dfOrdenado = pd.concat([dfOrdenado, dataframes[mes]], axis=0) #Y concateno cada dataframe

dfOrdenado = dfOrdenado.reset_index() #Reseteo los indices

# 3. Ventas por ciudades

In [ ]:
ciudades = []
for id in range(0 ,len(dfOrdenado)):
  ciudades.append(dfOrdenado['Dirección de Envio'][id].split(', ')[1])


dfOrdenado['Ciudades'] = ciudades

In [ ]:
mesdF = []
for id in range(0 ,len(dfOrdenado)):
  mesdF.append(dfOrdenado['Fecha de Pedido'][id].split('/')[0])


dfOrdenado['Meses'] = mesdF

In [ ]:
dfOrdenado

,index,ID de Pedido,Producto,Cantidad Pedida,Precio Unitario,Fecha de Pedido,Dirección de Envio,Ciudades,Meses
0,0,141234,iPhone,1,700,01/22/19 21:25,"944 Walnut St, Boston, MA 02215",Boston,01
1,1,141235,Lightning Charging Cable,1,14,01/28/19 14:15,"185 Maple St, Portland, OR 97035",Portland,01
2,2,141236,Wired Headphones,2,11,01/17/19 13:33,"538 Adams St, San Francisco, CA 94016",San Francisco,01
3,3,141237,27in FHD Monitor,1,149,01/05/19 20:33,"738 10th St, Los Angeles, CA 90001",Los Angeles,01
4,4,141238,Wired Headphones,1,11,01/25/19 11:59,"387 10th St, Austin, TX 73301",Austin,01
...,...,...,...,...,...,...,...,...,...
185945,25112,319666,Lightning Charging Cable,1,14,12/11/19 20:58,"14 Madison St, San Francisco, CA 94016",San Francisco,12
185946,25113,319667,AA Batteries (4-pack),2,3,12/01/19 12:01,"549 Willow St, Los Angeles, CA 90001",Los Angeles,12
185947,25114,319668,Vareebadd Phone,1,400,12/09/19 06:43,"273 Wilson St, Seattle, WA 98101",Seattle,12
185948,25115,319669,Wired Headphones,1,11,12/03/19 10:39,"778 River St, Dallas, TX 75001",Dallas,12


In [ ]:
ventasCiudades = dfOrdenado.groupby("Ciudades")["Cantidad Pedida"].sum() #Agrupo a cada ciudad la cantidad de ventas que se realizaron


fig = go.Figure()

fig.add_trace(go.Bar(
    x=ventasCiudades.keys(),
    y=ventasCiudades,
    marker=dict(color=ventasCiudades, colorscale = 'RdYlGn', colorbar=dict(title="Total de ventas", x=1.0, thickness=15)),  # Color y barra de colores
    text=ventasCiudades,
    textposition='auto',  # Muestro el valor encima de las barras

))

# Configuro los ejes y los títulos
fig.update_layout(
    title="Total de ventas por ciudades",
    xaxis=dict(title="Ciudad"),
    yaxis=dict(title="Total de ventas"),
)

fig.show()

In [ ]:
ventasCiudades

Ciudades
Atlanta          16602
Austin           11153
Boston           22528
Dallas           16730
Los Angeles      33289
New York City    27932
Portland         14053
San Francisco    50239
Seattle          16553
Name: Cantidad Pedida, dtype: int64

Georgia = Atlanta
Texas = Austin, Dallas
Massachusetts = Boston
California = Los Angeles, San Francisco
New york = new york city
Oregón = Portland
Washington = seattle

In [ ]:
estadosLista = ["Georgia", "Texas", "Massachusetts", "California", "New York", "Oregón", "Washington"]
estados = pd.Series()
estados["Georgia"] = ventasCiudades["Atlanta"]
estados["Texas"] = ventasCiudades["Austin"] + ventasCiudades["Dallas"]
estados["Massachusetts"] = ventasCiudades["Boston"]
estados["California"] = ventasCiudades["Los Angeles"] + ventasCiudades["San Francisco"]
estados["New York"] = ventasCiudades["New York City"]
estados["Oregón"] = ventasCiudades["Portland"]
estados["Washington"] = ventasCiudades["Seattle"]

fig = go.Figure()

fig.add_trace(go.Bar(
    x=estados.keys(),
    y=estados,
    marker=dict(color=estados, colorscale = 'RdYlGn', colorbar=dict(title="Total de ventas", x=1.0, thickness=15)),  # Color y barra de colores
    text=estados,
    textposition='auto',  # Muestro el valor encima de las barras

))

# Configuro los ejes y los títulos
fig.update_layout(
    title="Total de ventas por estado",
    xaxis=dict(title="Estado"),
    yaxis=dict(title="Total de ventas"),
)

fig.show()

<ipython-input-20-faa7dc51db1e>:2: FutureWarning:

The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.



In [ ]:

df_grouped = dfOrdenado.groupby(['Meses', 'Ciudades'])['Cantidad Pedida'].sum().unstack()

In [ ]:
fig = go.Figure()


for estado in df_grouped.columns:
    fig.add_trace(go.Bar(x=meses, y=df_grouped[estado], name=estado))


fig.show()

In [ ]:
type(ventasCiudades["Dallas"])

numpy.int64

Festivas


In [ ]:
ventasCiudades.keys()

Index(['Atlanta', 'Austin', 'Boston', 'Dallas', 'Los Angeles', 'New York City',
       'Portland', 'San Francisco', 'Seattle'],
      dtype='object', name='Ciudades')

In [ ]:
def extraer_datos(datos):
  datos["Fecha de Pedido"] = pd.to_datetime(datos["Fecha de Pedido"])
  datos["Hora"] = datos["Fecha de Pedido"].dt.hour
  datos["Dia"] = datos["Fecha de Pedido"].dt.day
  return datos



In [ ]:
VentasDia = {}

for mes in meses:
    dataframeMes = dataframes[mes]
    dataframeMes = extraer_datos(dataframeMes)  # Reemplaza esto con el procesamiento que necesites
    VentasDia[mes] = dataframeMes.groupby("Dia")["Cantidad Pedida"].count()

In [ ]:
fig = go.Figure()

diasLista = ["1 de Enero", "14 de Febrero","4 de Julio","17 de Marzo", "24 de Diciembre","25 de Diciembre"]

valores = pd.Series([VentasDia["Enero"].iloc[0], VentasDia["Febrero"].iloc[13], VentasDia["Marzo"].iloc[16], VentasDia["Julio"].iloc[3], VentasDia["Diciembre"].iloc[23] ,VentasDia["Diciembre"].iloc[24]] )

fig.add_trace(go.Bar(
    x=diasLista,
    y=valores,
    marker=dict(color=valores, colorscale = 'RdYlGn', colorbar=dict(title="Total de ventas", x=1.0, thickness=15)),  # Color y barra de colores
    text=valores,
    textposition='auto',  # Muestro el valor encima de las barras

))

# Configuro los ejes y los títulos
fig.update_layout(
    title="Total de ventas por dia Festivo",
    xaxis=dict(title="Ciudad"),
    yaxis=dict(title="Dia"),
)

fig.show()